In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.4 MB/s eta 0:00:00


In [ ]:
# !pip install datasets

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
# from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import torch

In [ ]:
# Load your dataset
dataset = pd.read_csv('/content/Fake-News-Telugu.csv')

dataset['Fact-check'] = dataset['Fact-check'].replace({'తప్పు': 0, 'తప్పుదోవ': 1, 'నిజం': 2})
# Split the data
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
valid_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

print(len(train_data),len(test_data),len(valid_data))
# Save the split datasets to separate files if needed
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)
valid_data.to_csv('valid_data.csv', index=False)


3638 455 455


In [ ]:
print(dataset['Fact-check'].value_counts()[0])
print(dataset['Fact-check'].value_counts()[1])
print(dataset['Fact-check'].value_counts()[2])

1906
736
1906


In [ ]:
print(test_data['Fact-check'].value_counts()[0])
print(test_data['Fact-check'].value_counts()[1])
print(test_data['Fact-check'].value_counts()[2])

189
75
191


In [ ]:
dataset.isnull().sum()

Claim               0
Fact                0
Fact-check          0
Reviewer            0
Date(YYYY-MM-DD)    0
Time                0
dtype: int64

In [ ]:
train_data.head()

,Claim,Fact,Fact-check,Reviewer,Date(YYYY-MM-DD),Time
540,2024 లో జరిగే లోక్ సభ ఎన్నికల్లో బీజేపీ మరోసార...,"కేంద్ర హోంమంత్రి, బీజేపీ అగ్రనేత అమిత్ షా శుక్...",2,HT Telugu Desk,2023-06-23,17:35:00
3948,బిజెపి ప్రభుత్వంలో మహిళలకి రక్షణ లేని కారణంగా ...,గొలుసులతో ఉన్న మహిళకి సంబంధించి మరొక ఫోటో ఒక వ...,0,Chaitanya,2020-10-01,18:19:08
2098,శ్రీలంకలో తవ్వకాల్లో బయటపడ్డ హనుమంతునికి చెంది...,ఈ గద భారతదేశంలోని ఇండోర్‌లోని ‘పిత్ర పర్వత్’ ఆ...,0,Varun Borugadda,2023-07-07,16:56:14
3280,తాజగా అఫ్గానిస్తాన్‌లో రాళ్లతో కొట్టి చంపేసిన ...,పోస్ట్ చేసిన ఫోటోలో ఉన్నది ఫర్ఖుండా మాలిక్జాదా...,0,Akhil Reddy,2021-08-19,17:43:13
4533,‘స్విస్ బ్యాంక్ కార్పొరేషన్’ వారు రాజీవ్ గాంధీ...,‘స్విస్ బ్యాంక్ కార్పొరేషన్’ 1998 లోనే ‘యూనియన...,0,Rakesh Vuppu,2019-09-13,16:46:53


In [ ]:
train_data["Claim"]

540     2024 లో జరిగే లోక్ సభ ఎన్నికల్లో బీజేపీ మరోసార...
3948    బిజెపి ప్రభుత్వంలో మహిళలకి రక్షణ లేని కారణంగా ...
2098    శ్రీలంకలో తవ్వకాల్లో బయటపడ్డ హనుమంతునికి చెంది...
3280    తాజగా అఫ్గానిస్తాన్‌లో రాళ్లతో కొట్టి చంపేసిన ...
4533    ‘స్విస్ బ్యాంక్ కార్పొరేషన్’ వారు రాజీవ్ గాంధీ...
                              ...                        
4426      అయోధ్యలో కట్టబోయే రామ మందిరం యొక్క డిజైన్ ఫోటో.
466     భారతదేశంలోనే కాదు.. దక్షిణాసియాలోనే కాదు.. ప్ర...
3092    ‘మహాత్మా గాంధీ స్వామి శ్రద్దానంద్‌ను చంపిన అబ్...
3772    మెర్సిడీస్ కారు తో పోలుస్తే ట్రాక్టర్ పై జీఎస్...
860     వరల్డ్ హ్యాపీనెస్ రిపోర్టు వెల్లడైంది. ఫిన్లాం...
Name: Claim, Length: 3638, dtype: object

In [ ]:
model_name = 'google/muril-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name, max_length=128)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels, max_length=None):
        self.encodings = encodings
        self.labels = labels
        self.max_length = max_length

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])

        if self.max_length is not None and 'input_ids' in self.encodings:
            item['input_ids'] = item['input_ids'][:self.max_length]
            item['attention_mask'] = item['attention_mask'][:self.max_length]

        return item

def encode_data(data, max_length=None):
    texts = list(data['Claim'])
    explanations = list(data['Fact'])

    flattened_claims = []
    for claim in texts:
        flattened_claims.append(str(claim))

    flattened_explanations = []
    for explanation in explanations:
        flattened_explanations.append(str(explanation))

    encodings = tokenizer(flattened_claims, flattened_explanations, truncation=True, padding='max_length', max_length=max_length)
    labels = data['Fact-check'].tolist()

    return CustomDataset(encodings, labels, max_length)


In [ ]:
# Prepare datasets
train_dataset = encode_data(train_data, max_length=512)
valid_dataset = encode_data(valid_data, max_length=512)
test_dataset = encode_data(test_data, max_length=512)  # No truncation for the test set

In [ ]:
batch_size = 8

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=100,
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    greater_is_better=False,
    weight_decay=0.01,
    warmup_steps=500,
    learning_rate=5e-5
)

In [ ]:
# Trainer for training and evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset
)

In [ ]:
# Train the model
trainer.train()

# Save the checkpoint
trainer.save_model("./saved_model_directory")

Epoch,Training Loss,Validation Loss
1,0.521500,0.361455
2,0.411100,0.357729
3,0.311400,0.302511


In [ ]:
# Evaluate on the test set
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = test_data['Fact-check'].tolist()

# Calculate performance metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')
class_report = classification_report(true_labels, predicted_labels)
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Classification Report:\n", class_report)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.8879120879120879
Precision: 0.8848151848151848
Recall: 0.8879120879120879
F1 Score: 0.8857273355046734
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.89      0.87       189
           1       0.70      0.61      0.65        75
           2       0.99      0.99      0.99       191

    accuracy                           0.89       455
   macro avg       0.85      0.83      0.84       455
weighted avg       0.88      0.89      0.89       455

Confusion Matrix:
 [[169  18   2]
 [ 29  46   0]
 [  0   2 189]]


In [ ]:
# Define class names for the labels (replace these with your actual class names)
class_names = ['Fake', 'Misleading', 'Not Fake']

# Displaying the confusion matrix as a heatmap with annotations and label names
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, cmap=plt.cm.Blues, interpolation='nearest')

plt.title('Confusion Matrix')
plt.colorbar()
plt.xticks(np.arange(len(class_names)), class_names)  # Set x-axis labels

# Set y-axis labels with rotation
yticks = plt.yticks(np.arange(len(class_names)), class_names)  # Set y-axis labels
for tick in yticks[1]:
    tick.set_rotation(90)  # Rotate labels vertically
    tick.set_horizontalalignment('center')  # Align labels at the center

plt.xlabel('Predicted labels')
plt.ylabel('True labels')

# Add text annotations
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        plt.text(j, i, str(conf_matrix[i, j]), horizontalalignment='center', verticalalignment='center', color='black')

plt.show()

In [ ]:
# Plot losses
train_losses = trainer.state.log_history['train_loss']
eval_losses = trainer.state.log_history['eval_loss']

plt.plot(train_losses, label='Training loss')
plt.plot(eval_losses, label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
!cp -r "/content/saved_model_directory" "/content/drive/MyDrive/DL-Fake-news-muril-3l-3e/"

In [ ]:
!cp -r "/content/drive/MyDrive/DL-Fake-news-muril-3l-3e/" "/content/saved_model_directory/"

In [ ]:
model_name = 'google/muril-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name, max_length=128)
model = AutoModelForSequenceClassification.from_pretrained("./saved_model_directory", num_labels=3)

In [ ]:
# Encode the test data
test_dataset = encode_data(test_data, max_length=None)

# Create a Trainer instance with the saved model and test dataset
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_dataset,
)

# Evaluate on the test set
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = test_data['Fact-check'].tolist()


# Calculate performance metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')
class_report = classification_report(true_labels, predicted_labels)
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print("Classification Report:\n", class_report)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.8879120879120879
Precision: 0.8848151848151848
Recall: 0.8879120879120879
F1 Score: 0.8857273355046734
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.89      0.87       189
           1       0.70      0.61      0.65        75
           2       0.99      0.99      0.99       191

    accuracy                           0.89       455
   macro avg       0.85      0.83      0.84       455
weighted avg       0.88      0.89      0.89       455

Confusion Matrix:
 [[169  18   2]
 [ 29  46   0]
 [  0   2 189]]


In [ ]:
model_name = 'google/muril-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name, max_length=128)
model = AutoModelForSequenceClassification.from_pretrained("./saved_model_directory", num_labels=3)

In [ ]:
class CustomTest(Dataset):
    def __init__(self, encodings, max_length=None):
        self.encodings = encodings
        self.max_length = max_length

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

        if self.max_length is not None and 'input_ids' in self.encodings:
            item['input_ids'] = item['input_ids'][:self.max_length]
            item['attention_mask'] = item['attention_mask'][:self.max_length]

        return item

def encode_test(data, max_length=None):
    texts = list(data['Claim'])
    explanations = list(data['Fact'])

    flattened_claims = []
    for claim in texts:
        flattened_claims.append(str(claim))

    flattened_explanations = []
    for explanation in explanations:
        flattened_explanations.append(str(explanation))

    encodings = tokenizer(flattened_claims, flattened_explanations, truncation=True, padding='max_length' if max_length else 'longest')
    return CustomTest(encodings, max_length)

In [ ]:
# Trainer for training and evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_dataset
)

In [ ]:
test = {  "Claim" : ["బీఆర్ఎస్ 18 స్థానాలకు మించి గెలిచే పరిస్థితి లేదని నివేదికలు అందటంతో కేసీఆర్ ప్రగతిభవన్ ఖాళీ చేస్తున్నారు అంటూ దిశ పత్రిక ప్రచురించింది"],
          "Fact"  : ["వైరల్ న్యూస్ ఆర్టికల్ కింద భాగంలో ఉన్న లింకును ఆధారంగా తీసుకొని వెతికితే, వేరే ఆర్టికల్‌కు దారి తీసింది.  పైగా దిశ న్యూస్ యాజమాన్యం వైరల్ పోస్టు ఫేక్ అంటూ తమ వెబ్సైట్ మరియు X పోస్టు ద్వారా తెలిపారు అని గమనించాం. కావున, పోస్టులో చేసిన క్లెయిమ్ తప్పు"]}

user_test = pd.DataFrame(test)

user_test_dataset = encode_test(user_test, max_length=None)

# Evaluate on the test set
predictions = trainer.predict(user_test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)


# For binary classification (fake or not)
if predicted_labels == 0:
    print("The model predicts this input is Fake.")
elif predicted_labels == 1:
    print("The model predicts this input is misleading.")
else:
    print("The model predicts this input is not Fake.")

The model predicts this input is Fake.


In [ ]:
test = {  "Claim" : ["నవంబర్ 25వ తేదీని నో నాన్ వెజ్ డే గా నిర్వహించాలని ఉత్తర ప్రదేశ్ ప్రభుత్వం నిర్ణయించింది. ఆ రోజు మాంసం అమ్మే దుకాణాలు, కబేళాలు మూసివేయాలని ఆదేశించింది."],
          "Fact"  : ["యోగి ఆదిత్యనాథ్ నేతృత్వంలోని ఉత్తరప్రదేశ్ ప్రభుత్వం నవంబర్ 25ని నో నాన్ వెజ్ డే గా ప్రకటించింది. సాధు టిఎల్ వాస్వానీ కి నివాళిగా నవంబర్ 25ని నాన్ వెజ్ డేగా ప్రకటించాలని యుపి ప్రభుత్వం నిర్ణయించింది. ప్రభుత్వ ఆదేశాల మేరకు.. ప్రతీ సంవత్సరం నవంబర్ 25వ తేదీన రోజు రాష్ట్రంలోని అన్ని కబేళాలు, మాంసం దుకాణాలు మూసివేయాల్సి ఉంటుంది. ఈ మేరకు ఉత్తరప్రదేశ్ ప్రభుత్వం సర్క్యులర్‌ జారీ చేసింది. ఈ ఆదేశాలను రాష్ట్రవ్యాప్తంగా కచ్చితంగా అమలయ్యేలా చూడాలని అధికారులను ఆదేశించింది."]}

user_test = pd.DataFrame(test)

user_test_dataset = encode_test(user_test, max_length=None)

# Evaluate on the test set
predictions = trainer.predict(user_test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)


# For binary classification (fake or not)
if predicted_labels == 0:
    print("The model predicts this input is Fake.")
elif predicted_labels == 1:
    print("The model predicts this input is misleading.")
else:
    print("The model predicts this input is not Fake.")

The model predicts this input is not Fake.


In [ ]:
test = {  "Claim" : ["పార్లమెంట్‌లో తెలంగాణ బిల్లు పాస్ కావడానికి 273 ఓట్లు కావాల్సి ఉండగా, కాంగ్రెస్-113, టీఆర్ఎస్- 2, బీజేపీ-185 వోట్లు వేయడం వల్ల తెలంగాణ బిల్లు పాస్ అయింది."],
          "Fact"  : ["తెలంగాణ ఏర్పాటుకు సంబంధించిన ఆంధ్రప్రదేశ్ పునర్వ్యవస్థీకరణ, 2014 బిల్లు పార్లమెంట్‌లో మూజువాణి ఓటుతో ఆమోదం పొందింది. ఈ బిల్లుపై ప్రత్యేకంగా వోటింగ్ జరగలేదు. ఐతే ఈ బిల్లుకు బీజేపీ మద్దతు తెలిపింది. కాగా 2009 జనరల్ ఎన్నికల్లో యుపిఎ కూటమి 262 సీట్లు గెలుచుకోగా, బీజేపీ నేతృత్వంలోని ఎన్డిఎ కూటమి 159 సీట్లు గెలుచుకుంది. కావున పోస్టు ద్వారా చెప్పేది తప్పుదోవ పట్టించే విధంగా ఉంది."]}

user_test = pd.DataFrame(test)

user_test_dataset = encode_test(user_test, max_length=None)

# Evaluate on the test set
predictions = trainer.predict(user_test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)


# For binary classification (fake or not)
if predicted_labels == 0:
    print("The model predicts this input is Fake.")
elif predicted_labels == 1:
    print("The model predicts this input is misleading.")
else:
    print("The model predicts this input is not Fake.")

The model predicts this input is misleading.
